## Cats vs Dogs
For this workshop you will be building a Convolutional neural network to classify cats vs dogs. You will need to be familiar with the theory of CNNs. Visit our lesson [here](http://caisplusplus.usc.edu/blog/curriculum/lesson7) for more info.

In [1]:
# Imports, make sure you have cv2 installed!
import os
import numpy as np
from scipy.ndimage import imread
from scipy.misc import imsave
import cv2
import sklearn.utils
import subprocess

# You are not allowed to change any of these constants.

INPUT_STRING = 'benbrooks'
DATA_PATH_1 = 'data_generation/positive_samples/'
DATA_PATH_2 = 'data_generation/false_samples/'
DATA_PATH_3 = 'data_generation/'
TEST_PERCENT = 0.2
SELECT_SUBSET_PERCENT = 1

# The cat and dog images are of variable size.
RESIZE_WIDTH=32
RESIZE_HEIGHT=32
EPOCHS = 5

In [2]:
# Lets get started by loading the data.
# Make sure you have the data downloaded to ./data
# To download the data go to https://www.kaggle.com/c/dogs-vs-cats/data and download train.zip

X = []
Y = []

DISPLAY_COUNT = 1000

files_1 = os.listdir(DATA_PATH_1)
for i, input_file in enumerate(files_1):
    img = imread(DATA_PATH_1 + input_file)
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    imsave('data_generation/t.' + str(i) + '.jpg', img)

files_2 = os.listdir(DATA_PATH_2)
for i, input_file in enumerate(files_2):
    img = imread(DATA_PATH_2 + input_file)
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    imsave('data_generation/f.' + str(i) + '.jpg', img)
    
subprocess.call(['rm', '-rf', 'data_generation/positive_samples'])
subprocess.call(['rm', '-rf', 'data_generation/false_samples'])
    
files_3 = os.listdir(DATA_PATH_3)
shuffled_files = sklearn.utils.shuffle(files_3)
print('Going to load %i files' % len(shuffled_files))

for i, input_file in enumerate(shuffled_files):
    if i % DISPLAY_COUNT == 0 and i != 0:
        print('Have loaded %i samples' % i)
        
    img = imread(DATA_PATH_3 + input_file)
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    X.append(img)
    if 't' == input_file.split('.')[0]:
        Y.append(1.0)
    else:
        Y.append(0.0)
        
X = np.array(X)
Y = np.array(Y)

test_size = int(len(X) * TEST_PERCENT)

test_X = X[:test_size]
test_Y = Y[:test_size]
train_X = X[test_size:]
train_Y = Y[test_size:]

print('Train set has dimensionality %s' % str(train_X.shape))
print('Test set has dimensionality %s' % str(test_X.shape))

# Apply some normalization here.
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X /= 255
test_X /= 255

Going to load 20000 files
Have loaded 1000 samples
Have loaded 2000 samples
Have loaded 3000 samples
Have loaded 4000 samples
Have loaded 5000 samples
Have loaded 6000 samples
Have loaded 7000 samples
Have loaded 8000 samples
Have loaded 9000 samples
Have loaded 10000 samples
Have loaded 11000 samples
Have loaded 12000 samples
Have loaded 13000 samples
Have loaded 14000 samples
Have loaded 15000 samples
Have loaded 16000 samples
Have loaded 17000 samples
Have loaded 18000 samples
Have loaded 19000 samples
Train set has dimensionality (16000, 32, 32, 3)
Test set has dimensionality (4000, 32, 32, 3)


In [3]:
######################################
#TODO: (Optional)
# Perform any data preprocessing steps



######################################

### Defining the network
Here are some useful resources to help with defining a powerful network.
- Convolution layers (use the 2D convolution) https://keras.io/layers/convolutional/
- Batch norm layer https://keras.io/layers/normalization/
- Layer initializers https://keras.io/initializers/
- Dense layer https://keras.io/layers/core/#dense
- Activation functions https://keras.io/layers/core/#activation
- Regulizers: 
    - https://keras.io/layers/core/#dropout
    - https://keras.io/regularizers/
    - https://keras.io/callbacks/#earlystopping
    - https://keras.io/constraints/

In [4]:
######################################
#TODO:
# Import necessary layers.
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.constraints import max_norm
######################################

model = Sequential()

######################################
#TODO:
# Define the network
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', input_shape=(RESIZE_WIDTH, RESIZE_HEIGHT, 3)))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512, kernel_initializer='glorot_normal'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='glorot_normal'))
model.add(Activation('sigmoid'))
######################################


######################################
#TODO:
# Define your loss and your objective
optimizer = RMSprop(lr=1e-4)
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
######################################

/Users/benbrooks/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Test Time
Now it's time to actually test the network. Don't change any of the parameters here except for the batch size.

Get above **65%**!

In [5]:
######################################
#TODO:
# Define the batch size
batch_size = 64
######################################
model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 12800 samples, validate on 3200 samples
Epoch 1/5
12800/12800 [==============================] - 52s 4ms/step - loss: 0.4722 - acc: 0.7649 - val_loss: 0.3094 - val_acc: 0.8447
Epoch 2/5
12800/12800 [==============================] - 50s 4ms/step - loss: 0.2244 - acc: 0.9088 - val_loss: 0.1582 - val_acc: 0.9363
Epoch 3/5
12800/12800 [==============================] - 47s 4ms/step - loss: 0.1286 - acc: 0.9557 - val_loss: 0.1020 - val_acc: 0.9650
Epoch 4/5
12800/12800 [==============================] - 47s 4ms/step - loss: 0.0906 - acc: 0.9698 - val_loss: 0.0836 - val_acc: 0.9747
Epoch 5/5
12800/12800 [==============================] - 46s 4ms/step - loss: 0.0713 - acc: 0.9774 - val_loss: 0.0730 - val_acc: 0.9788


In [6]:
loss, acc = model.evaluate(test_X, test_Y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))

4000/4000 [==============================] - 3s 800us/step

Got 98.00% accuracy


In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
predictions = model.predict(test_X)
for i in range(len(predictions)):
    if predictions[i] < 0.5:
        predictions[i] = 0.0
    else:
        predictions[i] = 1.0
    
print(classification_report(test_Y, predictions))
print(confusion_matrix(test_Y, predictions))

             precision    recall  f1-score   support

        0.0       0.99      0.97      0.98      1996
        1.0       0.97      0.99      0.98      2004

avg / total       0.98      0.98      0.98      4000

[[1932   64]
 [  16 1988]]
